In [ ]:
# !pip install git+https://github.com/XanaduAI/pennylane.git@prob_fn

In [ ]:
import pennylane as qml
from variational_notebooks.entangler import get_entangler_map
from pennylane import numpy as np
from pennylane.optimize import GradientDescentOptimizer,NesterovMomentumOptimizer
from pennylane.beta import prob


from data_loader import *


In [ ]:
idx_array = []
X = []
Y = []
labels = []
crops = []
sources = []
        
for idx, ((i, j), train, target, label, source, (cropX, cropY)) in enumerate(train_fashion_loader):
    
    idx_array.append((i, j))
    X.append(train.tolist())
    Y.append(target.item())
    labels.append(label.item())
    crops.append((cropX, cropY))
    
    sources.append(source)

In [ ]:
shirts_inds = [i for i in range(len(labels)) if labels[i]==6]
train_inds = shirts_inds[:100]
tests_inds = shirts_inds[100:200]

X_train = [X[i] for i in train_inds]
Y_train = [Y[i] for i in train_inds]

X_test = [X[i] for i in tests_inds]
Y_test = [Y[i] for i in tests_inds]

crops_test = [crops[i] for i in tests_inds]
sources_test = [sources[i] for i in tests_inds]

In [ ]:
len(Y_train),len(Y_test)

In [ ]:
n_wires = 9
n_qubits_readout = 8
dev = qml.device("default.qubit", wires=n_wires)

graph = get_entangler_map()

In [ ]:
nb_layer_application = 3

In [ ]:
# bitstring integer value
bitstring_values = list(range(2**n_qubits_readout))

In [ ]:
@qml.qnode(dev)
def quantum_completer(weights, x=None):
    
    for i in range(nb_layer_application):
    
        encoding(x)
        variational_circuit(weights[2*i],weights[2*i+1])

    return prob(wires=list(range(n_qubits_readout))) 

In [ ]:
# feature map 
# parameters should be of the form params = np.random.uniform(size=(num_layers, num_qubits, 3))

def encoding(x):
    for wire in range(n_wires // 3):
        qml.Rot(*x[3*wire: 3*(wire+1)], wires=[wire])     #Encode three features in each qubit

In [ ]:
# QAOA ansatz definition
# unitary operator U_B with parameter beta
def U_B(beta):
    for wire in range(n_wires):
        qml.RX(2 * beta, wires=wire)

# unitary operator U_C with parameter gamma
def U_C(gamma):
    for wire_pair in graph:
        qml.CNOT(wires=[wire_pair[0], wire_pair[1]])
        qml.RZ(2*gamma, wires=wire_pair[1])
        qml.CNOT(wires=[wire_pair[0], wire_pair[1]])
        
def variational_circuit(gamma,beta):

    U_C(gamma)
    U_B(beta)

In [ ]:
def loss(labels, predictions):
    return sum([(label - prediction)**2 for label, prediction in zip(labels, predictions)]) / float(len(labels))

def cost(weights, X, Y):
    predictions = [np.sum(quantum_completer(weights, x=x) * bitstring_values) / 255. for x in X]
    return loss(Y, predictions)

In [ ]:
def predict(X):
    return [np.mean(quantum_completer(weights, x=x) * bitstring_values) / 255. for x in X]

In [ ]:
%%time
# Define optimizer. 
opt = NesterovMomentumOptimizer(stepsize=0.01)
batch_size = 10

# Setup initial weights and number of optimization steps.
weights = np.random.random(2*nb_layer_application) 
steps = 10

# Optimizing the loss-function.
for it in range(3):
    # Update the weights by one optimizer step
    batch_index = np.random.randint(0, len(X_train), (batch_size,))
    X_batch = [X_train[i] for i in batch_index]
    Y_batch = [Y_train[i] for i in batch_index]
    weights = opt.step(lambda v: cost(v, X_batch, Y_batch), weights)
    
    print("Iter: {:5d} | Cost: {:0.7f} | Test: {:0.7f}".format(it + 1, cost(weights, X_train, Y_train),loss(predict(X_test), Y_hat_test)))

In [ ]:
Y_hat_test = predict(X_test)
loss(Y_test, Y_hat_test)

In [ ]:
# ind_example = 0

# X_test_example = X_test[ind_example]
# source_test_example = np.array(sources_test[ind_example].tolist())
# cropX, cropY = crops_test[ind_example]

# new_window_values = np.array(X_test_example).reshape(3, 3)
# # source_test_example[cropY:cropY+3, cropX:cropX+3] = new_window_values

# fig = plt.figure()
# ax = fig.add_subplot(111)
# rect = patches.Rectangle((cropX-1, cropY-1),4,4,linewidth=1,edgecolor='r',facecolor='none')
# ax.add_patch(rect)
# ax.imshow(source_test_example, cmap=plt.cm.gray, vmin=0, vmax=1)
# fig.savefig(f'./images/full_{idx}.png')

# source_test_example[cropY:cropY+3, cropX:cropX+3] = new_window_values

# fig = plt.figure()
# ax = fig.add_subplot(111)
# rect = patches.Rectangle((cropX-1, cropY-1),4,4,linewidth=1,edgecolor='r',facecolor='none')
# ax.add_patch(rect)
# ax.imshow(source_test_example, cmap=plt.cm.gray, vmin=0, vmax=1)
# fig.savefig(f'./images/full_{idx}_crop.png')